# Projeto 1 - Ciência dos Dados

Nome: Luiz Ricardo Hardman Paranhos

Nome: Matheus Kwon

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [3]:
import os

filename = 'cheetos.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo cheetos.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,relevancia
0,@eaipaixao_ cheetos obv,2
1,@giovannapinto mano eu fui comprar 2 cheetos e...,2
2,@zabele__ só tenho cheetos,1
3,"@lilizvante cheetos, old\nhttps://t.co/atamf2woj7",1
4,@biaventura14 kkkkkkkkkkkkkkkkkkkkk cheetos ve...,2


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,relevancia
0,sou do tipo que quando tá conversando com algu...,0
1,att fresquinha de flaming hot cheetos!!!! http...,1
2,me entupindo de cheetos enquanto vejo tapas e ...,1
3,cheetos ou doritos https://t.co/caytqjylnp,2
4,mas quem colocou o cheetos na minha tml hoje ?,2


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

- Relevante: informação da qual a empresa pode ganhar dinheiro
- Neutro: informações sobre o salgadinho cheetos, mas que não necessariamente interessa à empresa
- Irrelevante: resto

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
train['relevancia'] = train['relevancia'].astype('category')
train.relevancia.cat.categories = ['irrelevante', 'neutro', 'relevante']

In [7]:
# https://docs.python.org/3/library/re.html#
import functools
import operator
import re
import emoji


def cleanup(text):
    # Função de limpeza muito simples que troca alguns sinais básicos por espaços
    # import string
    punctuation = '[!-.:?;/,_@¿]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def cleanup_enter(text):
    punctuation = '\n'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

def deEmojify(text):
    # retirado do stackoverflow
    em_split_emoji = emoji.get_emoji_regexp().split(text)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    p = ''
    for separated in em_split:
        p += separated + ' '
    return p
#    regrex_pattern = re.compile(pattern = "["
#        u"\U0001F600-\U0001F64F"  # emoticons
#        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#        u"\U0001F680-\U0001F6FF"  # transport & map symbols
#        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags = re.UNICODE)
#    return regrex_pattern.sub(r'',text)
    

def link(text):
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\S+', '', text, flags=re.MULTILINE)
    return text

In [8]:
# String com todas as palavras
tweets_treinamento_raw = ''
for tweet in train.Treinamento:
    tweets_treinamento_raw = tweets_treinamento_raw + ' ' + tweet
tweets_treinamento = deEmojify(cleanup_enter(cleanup(link(tweets_treinamento_raw.lower()))))

In [9]:
# String com os relevantes
filtro_relevante_train = train.relevancia == 'relevante'
treinamento_relevante = train.loc[filtro_relevante_train,:]

tweets_relevantes = ''
for tweet in treinamento_relevante.Treinamento:
    tweets_relevantes = tweets_relevantes + ' ' + tweet

treinamento_relevante = deEmojify(cleanup_enter(cleanup(link(tweets_relevantes.lower()))))

In [10]:
# String com os irrelevantes
filtro_irrelevante_train = train.relevancia == 'irrelevante'
treinamento_irrelevante = train.loc[filtro_irrelevante_train,:]

tweets_irrelevantes = ''
for tweet in treinamento_irrelevante.Treinamento:
    tweets_irrelevantes = tweets_irrelevantes + ' ' + tweet

    
treinamento_irrelevante = deEmojify(cleanup_enter(cleanup(tweets_irrelevantes.lower())))

In [11]:
# String com os neutros
filtro_neutro_train = train.relevancia == 'neutro'
treinamento_neutro = train.loc[filtro_neutro_train,:]

tweets_neutro = ''
for tweet in treinamento_neutro.Treinamento:
    tweets_neutro = tweets_neutro + ' ' + tweet

    
treinamento_neutro = deEmojify(cleanup_enter(cleanup(tweets_neutro.lower())))

In [12]:
# Criando lista e séries a partir das strings
tweets_treinamento_lista = tweets_treinamento.split()
treinamento_relevante_lista = treinamento_relevante.split()
treinamento_irrelevante_lista = treinamento_irrelevante.split()
treinamento_neutro_lista = treinamento_neutro.split()


serie_treinamento = pd.Series(tweets_treinamento_lista)
serie_treinamento_relevante = pd.Series(treinamento_relevante_lista)
serie_treinamento_irrelevante = pd.Series(treinamento_irrelevante_lista)
serie_treinamento_neutro = pd.Series(treinamento_neutro_lista)

In [13]:
tabela_treinamento = serie_treinamento.value_counts()
tabela_treinamento

cheetos    297
de         171
e          116
um          81
o           81
          ... 
ponto        1
veio         1
formato      1
find         1
zero         1
Length: 1261, dtype: int64

In [14]:
tabela_treinamento_relevante = serie_treinamento_relevante.value_counts()
tabela_treinamento_relevante

cheetos      86
de           49
e            23
requeijão    19
o            18
             ..
🤯             1
my            1
ja            1
sensação      1
frita         1
Length: 287, dtype: int64

In [15]:
stopwords = ['por', 'se','uma','em','mais','q','que','na','da','do','no','eu','de','e','o','um','a','com','pra', 'mas', 'só', 'so']
def remove_stopwords(lista):
    nova_lista = []
    for palavra in lista:
        x = 0
        if palavra in stopwords:
            x = 1
        if x == 0:
            # transformando verbos no gerúndio em verbos no infinitivo
            if palavra[-3:] == 'ndo' or palavra[-3:] == 'mos':
                if palavra == 'quando' or palavra == 'mando':
                    nova_lista.append(palavra)
                else:
                    nova_lista.append(palavra[:-3]+'r')
            elif palavra == 'n' or palavra == 'nao':
                nova_lista.append('não')
            elif palavra == 'ta' or palavra =='tá':
                nova_lista.append('está')
            else:
                nova_lista.append(palavra)
    return nova_lista

In [16]:
tweets_treinamento_stopwords_lista = remove_stopwords(tweets_treinamento_lista)
        

serie_treinamento_filtrado = pd.Series(tweets_treinamento_stopwords_lista)
tabela_treinamento_filtrado = serie_treinamento_filtrado.value_counts()
tabela_treinamento_filtrado_relativa = serie_treinamento_filtrado.value_counts(True)
tabela_treinamento_filtrado.head(20)

cheetos      297
é             60
não           50
comer         38
requeijão     31
doritos       26
tem           26
meu           23
me            19
gosto         14
pq            13
melhor        13
vou           12
minha         12
cheiro        12
vontade       12
queijo        11
está          11
como          11
pacote        10
dtype: int64

In [17]:
# Removendo stop words da lista de todas as palavras relevantes
treinamento_relevante_stopwords_lista = remove_stopwords(treinamento_relevante_lista)

serie_treinamento_relevante_filtrado = pd.Series(treinamento_relevante_stopwords_lista)
tabela_treinamento_relevante_filtrado = serie_treinamento_relevante_filtrado.value_counts()
tabela_treinamento_relevante_filtrado_relativa = serie_treinamento_relevante_filtrado.value_counts(True)
tabela_treinamento_relevante_filtrado.head(20)
# palavras_relevantes_totais = len(tabela_treinamento_relevante_filtrado)
# primeiras_relevantes = tabela_treinamento_relevante_filtrado.head(20)
# P_Palavras = {}
# for palavras, valor in primeiras_relevantes.items():
#     P_Palavras[palavra] = (valor + 1) / (palavras_totais + palavras_relevantes_totais)
# print(P_Palavras)

cheetos       86
requeijão     19
é             18
doritos       16
comer         13
melhor        11
vontade        9
fandangos      8
queria         7
gosto          7
😋              6
sabor          5
não            5
bom            5
pq             4
eh             4
salgadinho     4
🤤              4
ruim           4
nada           3
dtype: int64

In [18]:
# Removendo stop words da lista de todas as palavras irrelevantes
treinamento_irrelevante_stopwords_lista = remove_stopwords(treinamento_irrelevante_lista)

serie_treinamento_irrelevante_filtrado = pd.Series(treinamento_irrelevante_stopwords_lista)
tabela_treinamento_irrelevante_filtrado = serie_treinamento_irrelevante_filtrado.value_counts()
tabela_treinamento_irrelevante_filtrado_relativa = serie_treinamento_irrelevante_filtrado.value_counts(True)
tabela_treinamento_irrelevante_filtrado.head(20)

cheetos    116
não         32
é           24
tem         13
me          11
meu         11
cheiro      10
comer        8
está         8
minha        7
seu          7
vou          7
ele          7
pq           6
tipo         6
aqui         6
como         6
pelo         5
estar        5
😭            5
dtype: int64

In [19]:
treinamento_neutro_stopwords_lista = remove_stopwords(treinamento_neutro_lista)

serie_treinamento_neutro_filtrado = pd.Series(treinamento_neutro_stopwords_lista)
tabela_treinamento_neutro_filtrado = serie_treinamento_neutro_filtrado.value_counts()
tabela_treinamento_neutro_filtrado_relativa = serie_treinamento_neutro_filtrado.value_counts(True)
tabela_treinamento_neutro_filtrado.head(20)

cheetos       95
é             18
comer         17
não           13
tem           11
requeijão     10
meu            9
me             8
doritos        7
as             6
1              5
pacote         5
gosto          5
comprar        4
comi           4
mim            4
tudo           4
salgadinho     4
biscoito       4
café           4
dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [20]:
P_R = len(treinamento_relevante_stopwords_lista) / len(tweets_treinamento_stopwords_lista)
P_Rc = len(treinamento_irrelevante_stopwords_lista) / len(tweets_treinamento_stopwords_lista)
P_N = len(treinamento_neutro_stopwords_lista) / len(tweets_treinamento_stopwords_lista)
classificador = []
def testar (tweet):
    P_tweetDadoR = 1
    P_tweetDadoRc = 1
    P_tweetDadoN = 1
    for palavra in tweet_teste:
        # verifica se a palavra existe no relevante
        if palavra in tabela_treinamento_relevante_filtrado:
            P_tweetDadoR = P_tweetDadoR*(tabela_treinamento_relevante_filtrado[palavra]+1) / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_relevante_filtrado))
            # verifica se a palavra existe no irrelevante e no relevante
            if palavra in tabela_treinamento_irrelevante_filtrado:
                P_tweetDadoRc = P_tweetDadoRc*(tabela_treinamento_irrelevante_filtrado[palavra]+1) / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_irrelevante_filtrado))
                # verifica se a palavra existe no irrelevante, neutro e relevante
                if palavra in tabela_treinamento_neutro_filtrado:
                    P_tweetDadoN = P_tweetDadoN*(tabela_treinamento_neutro_filtrado[palavra]+1) / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_neutro_filtrado))
                # se não existe a palavra no neutro, considera-se frequência 0
                else:
                    P_tweetDadoN = P_tweetDadoN / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_neutro_filtrado))
            # se não existe a palavra no irrelevante, considera-se frequencia 0
            else:
                P_tweetDadoRc = P_tweetDadoRc / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_irrelevante_filtrado)) 
                 # verifica se a palavra existe no neutro e relevante
                if palavra in tabela_treinamento_neutro_filtrado:
                    P_tweetDadoN = P_tweetDadoN*(tabela_treinamento_neutro_filtrado[palavra]+1) / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_neutro_filtrado))
                # se não existe a palavra no neutro, considera-se frequência 0
                else:
                    P_tweetDadoN = P_tweetDadoN / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_neutro_filtrado))
        else:
            # se n existe a palavra no relevante, considera-se frequência 0
            P_tweetDadoR = P_tweetDadoR / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_relevante_filtrado))
             # verifica se a palavra existe no irrelevante
            if palavra in tabela_treinamento_irrelevante_filtrado:
                P_tweetDadoRc = P_tweetDadoRc*(tabela_treinamento_irrelevante_filtrado[palavra]+1) / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_irrelevante_filtrado))
                 # verifica se a palavra existe no irrelevante e neutro 
                if palavra in tabela_treinamento_neutro_filtrado:
                    P_tweetDadoN = P_tweetDadoN*(tabela_treinamento_neutro_filtrado[palavra]+1) / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_neutro_filtrado))
                # se não existe a palavra no neutro, considera-se frequência 0
                else:
                    P_tweetDadoN = P_tweetDadoN / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_neutro_filtrado))

            # se n existe a palavra no irrelevante, considera-se frequência 0
            else:
                P_tweetDadoRc = P_tweetDadoRc / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_irrelevante_filtrado))
                 # verifica se a palavra existe no neutro
                if palavra in tabela_treinamento_neutro_filtrado:
                    P_tweetDadoN = P_tweetDadoN*(tabela_treinamento_neutro_filtrado[palavra]+1) / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_neutro_filtrado))
                # se não existe a palavra no neutro, considera-se frequência 0
                else:
                    P_tweetDadoN = P_tweetDadoN / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_neutro_filtrado))
    P_RDadoTweet = P_tweetDadoR * P_R
    P_RcDadoTweet = P_tweetDadoRc * P_Rc
    P_NdadoTweet = P_tweetDadoN * P_N
    if P_RDadoTweet >= P_RcDadoTweet:
        if P_RDadoTweet >=  P_NdadoTweet:
            classificador.append('relevante')
        else:
            classificador.append('neutro')
    else:
        if P_RcDadoTweet > P_NdadoTweet:
            classificador.append('irrelevante')
        else:
            classificador.append('neutro')

for tweet in test.Teste:
    tweet_teste = remove_stopwords(deEmojify(cleanup_enter(cleanup(link(tweet.lower())))).split())
    testar(tweet_teste)
test['classificador'] = classificador
test  

,Teste,relevancia,classificador
0,sou do tipo que quando tá conversando com algu...,0,neutro
1,att fresquinha de flaming hot cheetos!!!! http...,1,neutro
2,me entupindo de cheetos enquanto vejo tapas e ...,1,irrelevante
3,cheetos ou doritos https://t.co/caytqjylnp,2,relevante
4,mas quem colocou o cheetos na minha tml hoje ?,2,irrelevante
...,...,...,...
195,"@bebediaboo eu muito li ""anti-cheetos""",0,relevante
196,"salgadinho, cheetos, biscoito recheado, biscoi...",0,neutro
197,queria experimentar takis e hot cheetos,1,neutro
198,@lwttersondre cheetos de requeijão &gt;&gt;&gt...,2,relevante


In [23]:
sentimento = pd.crosstab(test.relevancia, test.classificador, normalize='columns')
sentimento

classificador,irrelevante,neutro,relevante
relevancia,,,
0,0.573529,0.362069,0.229730
1,0.279412,0.362069,0.189189
2,0.147059,0.275862,0.581081


In [37]:
# Relatório resumo

# Verdadeiros resultados

print (f'Verdadeiros irrelevantes: {(sentimento.irrelevante[0]*100).round(3)}%')
print (f'Verdadeiros neutros: {(sentimento.neutro[1]*100).round(3)}%')
print (f'Verdadeiros relevantes: {(sentimento.relevante[2]*100).round(3)}%')

print('-----------------------------------------------------------------------')

# Falsos resultados

print (f'Falsos irrelevantes: {((sentimento.irrelevante[1] + sentimento.irrelevante[2])*100).round(3)}%')
print (f'Falsos neutros: {((sentimento.neutro[2] + sentimento.neutro[1])*100).round(3)}%')
print (f'Falsos relevantes: {((sentimento.relevante[1] + sentimento.relevante[0])*100).round(3)}%')


print('-----------------------------------------------------------------------')

# Quantas vezes nosso classificador acertou

print(f'Acertou em: {(sentimento.irrelevante[0] + sentimento.relevante[2] + sentimento.irrelevante[0])}')

Verdadeiros irrelevantes: 57.353%
Verdadeiros neutros: 36.207%
Verdadeiros relevantes: 58.108%
-----------------------------------------------------------------------
Falsos irrelevantes: 42.647%
Falsos neutros: 63.793%
Falsos relevantes: 41.892%


___
### Concluindo

#### Sobre o classificador:
- O nosso classificador tem uma performance razoável para classificar tweets relevantes e irrelevantes. No entanto, dentre os tweets classificados como neutros, há uma dúvida maior em se ele realmente é neutro ou se ele é, na verdade, irrelevante.

#### Como são tratadas mensagens de dupla negação ou sarcasmo:
- Quanto aos tweets que contém dupla negação e sarcasmo, nosso classificador funcionará na maioria das vezes, pois tweets que contém palavras de negação ou sarcásticas não afetarão os conceitos classificatórios de nosso classificador, visto que os nossos conceitos de relevância são baseados na condição de demonstração de opinião. Ou seja, caso o tweet esteja relacionado a alguma forma de opinião, ele será relevante. 

#### Sugestões de melhorias (plano de expansão):
- Uma sugestão de melhoria do classificador seria separar, dentre os tweets relevantes, os que falam positivamente do produto dos que falam negativamente, porque assim é possível ver potenciais melhorias no produto e também fatores a serem mantidos nele. Para implementar essa mudança, pensamos em criar uma base de dados separada dos tweets relevantes, eliminando os neutros e irrelevantes das probabilidades.

- Também pensamos na utilização de bigramas, ou até de trigramas (a probabilidade de 2 ou 3 palavras estarem juntas), método que pode aumentar a eficácia do classificador, pois considera a combinação das palavras, já que existe um contexto na ocorrência de cada palavra. Essa melhoria pode diminuir a dúvida dos tweets neutros do nosso classificador. Isso poderia ser realizado criando, a partir dos próprios tweets, tabelas das combinações de duas/três palavras consecutivas como cada elemento delas. (Fonte: [Naive Bayes and Text Classification I](https://arxiv.org/pdf/1410.5329.pdf) )

- Outra melhoria que sugerimos é o uso de probabilidades logarítmicas para diminuir a parte decimal das probabilidades. Efetivo, pois, quando a parte decimal delas é muito extensa, além de haver risco de ocorrer "overflow", a combinação delas fica absurdamente pequena, dificultando a comparação entre probabilidades. Essa melhoria pode diminuir a dúvida dos tweets neutros do nosso classificador. Isso pode ser feito ao utilizar o logaritmo das frequências para obter as probabilidades. (Fonte: [Naive Bayes and Text Classification I](https://arxiv.org/pdf/1410.5329.pdf) )

- Sendo assim, para realizar nosso plano de expansão, seria necessária uma maior captação de investimentos para então viabilizar as melhorias computacionais necessárias, visto que o método Naive-Bayes de classificação é caracterizado como "eager learner", demandando assim uma capacidade computacional maior que outros métodos "lazy learners".

#### Não podemos utilizar o classificador para gerar mais amostras de treinamento:
- Caso fosse decidido por utilizar nosso classificador para gerar uma amostra maior de treinamento, o nosso classificador se tornaria enviesado, pois quando desejamos por obter uma base de dados maior, o nosso objetivo seria por aperfeiçoar as probabilidades de relevância de cada palavra, classificando manualmente. No entanto, caso utilizássemos o nosso classificador para gerar mais tweets, a relevância de cada palavra não seria necessariamente a correta, visto que nem sempre o classificador atribui a relevância desejada, observado no teste de sentimento.

#### Naive-Bayes em contextos diferentes do projeto:
- Classificação de documentos (artigos, reportagens ou blogs), separando os tipos de produção textual, por meio das especificidades ortográficas e de design de cada um dos diferentes tipos de reportagem. Por exemplo, no caso de uma revista, tem-se uma ortografia mais informal e ligada a um público-alvo. Já um artigo científico, tem-se o design dividido em tópicos comum a todos os artigos, assim como uma linguagem bastante formal e científica.
- Classificação de poemas com base nas especificidades de escrita de cada escola literária. Por exemplo, no arcadismo, era bastante comum a utilização da frase: "carpe diem". Também classificaremos com base no design do poema. Por exemplo, no parnasianismo, o adjetivo era utilizado em demasia, já na segunda geração do romantismo, palavras em tons depressivos ou que remetem à tristeza, eram os mais comumente utilizados.
- Também é possível utilizar o Naive-Bayes no contexto de classifição de fake news. Pode-se fazer isso através da análise de alguns conceitos que pertencem à quase todas notícias falsas, como a falta de uma fonte confiável ou a utização de palavras bastante chamativas em demasia.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* CHECK  Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* CHECK  Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* CHECK  Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* CHECK  Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* CHECK  Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* CHECK  Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* CHECK  Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**